In [81]:
import pandas as pd
import numpy as np
import string
from sklearn.naive_bayes import GaussianNB
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split

import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer

In [82]:
data = pd.read_csv("spam.csv")

In [83]:
data.shape

(5171, 4)

In [84]:
data.head(5)

,Unnamed: 0,label,text,label_num
0,605,ham,Subject: enron methanol ; meter # : 988291\r\n...,0
1,2349,ham,"Subject: hpl nom for january 9 , 2001\r\n( see...",0
2,3624,ham,"Subject: neon retreat\r\nho ho ho , we ' re ar...",0
3,4685,spam,"Subject: photoshop , windows , office . cheap ...",1
4,2030,ham,Subject: re : indian springs\r\nthis deal is t...,0


In [85]:
nltk.download("stopwords")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [86]:
data.drop(["Unnamed: 0"],axis =1,inplace = True)

In [87]:
data.head()

,label,text,label_num
0,ham,Subject: enron methanol ; meter # : 988291\r\n...,0
1,ham,"Subject: hpl nom for january 9 , 2001\r\n( see...",0
2,ham,"Subject: neon retreat\r\nho ho ho , we ' re ar...",0
3,spam,"Subject: photoshop , windows , office . cheap ...",1
4,ham,Subject: re : indian springs\r\nthis deal is t...,0


In [88]:
data.text = data["text"].apply(lambda x: x.replace('\r\n' ," "))

In [89]:
data['text'].iloc[0]

"Subject: enron methanol ; meter # : 988291 this is a follow up to the note i gave you on monday , 4 / 3 / 00 { preliminary flow data provided by daren } . please override pop ' s daily volume { presently zero } to reflect daily activity you can obtain from gas control . this change is needed asap for economics purposes ."

In [90]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5171 entries, 0 to 5170
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   label      5171 non-null   object
 1   text       5171 non-null   object
 2   label_num  5171 non-null   int64 
dtypes: int64(1), object(2)
memory usage: 121.3+ KB


In [91]:
stemmer = PorterStemmer()
# stemmer.stem("panda") ##replace the words in shorter
corpus = []
stopwords_set  = set(stopwords.words("english"))
for i in range(len(data)):
  text = data["text"].iloc[i].lower()
  text = text.translate(str.maketrans("","",string.punctuation)).split()
  text = [stemmer.stem(word) for word in text  if word not in stopwords_set]
  text = " ".join(text)
  corpus.append(text)

In [92]:
corpus[0]

'subject enron methanol meter 988291 follow note gave monday 4 3 00 preliminari flow data provid daren pleas overrid pop daili volum present zero reflect daili activ obtain ga control chang need asap econom purpos'

In [93]:
#vectorize the text
vectorizer = CountVectorizer()
x = vectorizer.fit_transform(corpus).toarray()
y = data["label_num"]

In [94]:
x_train , x_test ,y_train ,y_test = train_test_split(x,y ,test_size =0.26,random_state =10)

In [95]:
print(f'''
len of x_train:         {len(x_train)}
len of x_test:          {len(x_test)}
len of y_train:         {len(y_train)}
len of y_test:          {len(y_test)}
      ''')

  
len of x_train:         3826
len of x_test:          1345
len of y_train:         3826       
len of y_test:          1345
      


In [96]:
model = GaussianNB()
model.fit(x_train,y_train)

GaussianNB()

In [97]:
y_pred  = model.predict(x_test)
y_pred

array([0, 1, 1, ..., 0, 0, 1])

In [98]:
from sklearn.metrics import accuracy_score
print()
accuracy_score(y_test,y_pred)*100


95.68773234200744

In [99]:
n = input()
n_vec = vectorizer.transform([n]).toarray()
y_predicted = model.predict(n_vec)
print(y_predicted)

hello this is suvashis panda
[0]


In [100]:
if y_predicted == 0:
  print("it's genuine")
else:
  print("it's spam ")

it's genuine


In [101]:
def predict(n):
     stemmer = PorterStemmer()
     stopwords_set  = set(stopwords.words("english"))
     n_vec = n.lower().translate(str.maketrans('','',string.punctuation)).split()
     n_vec = [stemmer.stem(word) for word in n_vec  if word not in stopwords_set]
     # Join the stemmed words back into a single string to represent the document
     corpus = [" ".join(n_vec)]
     x_email = vectorizer.transform(corpus)
     x_email_dense = x_email.toarray()
     y_predicted = model.predict(x_email_dense)
     if y_predicted == 0:
       print("it's genuine")
     else:
       print("it's spam ")

In [102]:
mail = input()
predict(mail)

suggest this off
it's genuine
